# Getting Started

In this tutorial, we'll learn how to simulate probabilities with ``pathfinder2e_stats``.

To follow this tutorial, you'll need to have at least a basic understanding of
- The [Pathfinder rules](https://2e.aonprd.com/PlayersGuide.aspx), and
- Data science workflows, e.g. based on Python + pandas + Jupyter notebooks. See {ref}`audience`.

If you don't have your Jupyter Notebook development environment ready yet, go back to {doc}`../installing`.

## Rolling some dice
Let's start simple - let's import the module and roll a d6.

We're going to roll it *one hundred thousand times.*

In [ ]:
import pathfinder2e_stats as pf2

oned6 = pf2.roll(1, 6)
oned6

``pathfinder2e_stats`` functions return standard {class}`xarray.DataArray` and {class}`xarray.Dataset` objects, which can be analyzed with standard data science techniques. We can start immediately answering some questions - for example, what is the mean roll?

In [ ]:
oned6.mean()

Note that the result above is a *numerical approximation*: the mean of rolling 1d6 an *infinite* amount of times is *exactly* 3.5. If we roll it less times than that, however, there's going to be some error.

If we roll it again, we are going to get a different sequence. This is because ``pathfinder2e_stats`` uses a global random number generator, which by default is reset to a fixed seed every time you restart your notebook. See {func}`~pathfinder2e_stats.seed`.

In [ ]:
pf2.roll(1, 6)

Well, that was easy, but we could have figured out the answer by doing the maths on the back of an envelope! Let's move on to something that is more complicated. A timeless classic: a 6d6 {prd_spells}`Fireball <1530>`!

In [ ]:
fireball = pf2.roll(6, 6)
fireball

What is the damage distribution? First we're going to calculate it numerically; then we'll visualize it with ``matplotlib`` (but we could use any other library, like ``plotly`` or ``hvplot``).

In [ ]:
fireball.value_counts("roll").to_pandas()

In [ ]:
_ = fireball.to_pandas().hist(bins=30)

But wait - that's just the base damage! The *actual* damage of a fireball depends on the target's reflex saving throw, as well as their resistances, immunities and weaknesses. ``pathfinder2e_stats`` makes dealing with all this very easy.

## Rolling checks

In Pathfinder, a *check* is whenever one rolls a d20+bonus against a DC; this includes attack rolls against AC.

For example, a paladin with +8 Diplomacy tries to convince a guard to let them pass. The DC is 15.
To simulate that, we call {func}`~pathfinder2e_stats.check`.

In [ ]:
request = pf2.check(8, DC=15)
request

The output of {func}`~pathfinder2e_stats.check` is a Dataset, which contains several variables. We normally only care about the last one, `outcome`. However, there are several other variables before it that explain *how* we reached that outcome, allowing us to fully trace its logic:

- **natural** is the bare d20 roll, 100,000 times
- **outcome** is the roll's degree of success, taking into account critical success/failure rules, natural 1s and 20s.

`outcome` is an integer (sadly there are no categorical dtypes in xarray yet), whose meaning is mapped in the `legend` attribute of the dataset, as shown above. It is also available in the {class}`~pathfinder2e_stats.DoS` enum. For the sake of robustness and readability, when you express an outcome (we'll see later when and how) you should always use `DoS` and never its numerical value.

Have a look at the {func}`API documentation <pathfinder2e_stats.check>` for additional parameters, such as fortune/misfortune effects to roll twice and take highest/lowest, conditionally using hero points depending on initial outcome and special rules like the {prd_equipment}`Keen <2843>` rune.

You can aggregate the result by using handy helpers such as {func}`~pathfinder2e_stats.outcome_counts`:

In [ ]:
# Probability to get each outcome
pf2.outcome_counts(request).to_pandas()

It's also common to compare against `DoS`. Operators `>`, `>=`, `<`, and `<=` are supported:

In [ ]:
# Probability to get at least a success
(request.outcome >= pf2.DoS.success).mean().item()

In [ ]:
# Probability to get a critical success
(request.outcome == pf2.DoS.critical_success).mean().item()

Attack rolls, saving throws, counteract checks, flat checks, etc. work exactly in the same way as skill checks.
For example, the party rogue can Strike a bandit (AC22) with his +14 rapier:

In [ ]:
strike = pf2.check(14, DC=22)
pf2.outcome_counts(strike).to_pandas()

Or a wizard can blast the bandit, who has +10 reflex, with his DC21 fireball:

In [ ]:
save = pf2.check(10, DC=21)
pf2.outcome_counts(save).to_pandas()

Finally, with {func}`~pathfinder2e_stats.map_outcome` you can post-process the check outcome, for example to define the Evasion class feature or similar (if you roll a success, you get a critical success instead):

In [ ]:
save_with_evasion = pf2.map_outcome(save, evasion=True)
pf2.outcome_counts(save_with_evasion).to_pandas()

## Damage profiles

Previously, we saw how to roll raw 6d6. However, let's refine that - let's define the *damage profile* of a fireball, which we're going to roll in the next section.

In [ ]:
fireball = pf2.Damage("fire", 6, 6, basic_save=True)
fireball

Damage offers many keyword arguments and supports addition.
Let's have a rogue's 2d8+3 deadly d8 rapier, with 1d6 sneak attack:

In [ ]:
rapier = pf2.Damage("piercing", 2, 6, 3, deadly=8)
sneak_attack = pf2.Damage("precision", 1, 6)
rapier + sneak_attack

When rolling damage in the next chapter, we'll see that `pathfinder2e_stats` automatically manages the deadly, fatal, etc. traits.
To preview the breakdown of what's going to be rolled for each degree of success, we can call the {meth}`~pathfinder2e_stats.Damage.expand` method:

In [ ]:
(rapier + sneak_attack).expand()

Note how the `basic_save=True` flag on the fireball damage profile means it expands differently from a weapon:

In [ ]:
fireball.expand()

If basic save/basic attack damage rules, deadly, fatal, etc. are not enough, it's possible to hand-craft more sophisticated damage profiles with {class}`~pathfinder2e_stats.ExpandedDamage` - which is what you get when you call {meth}`~pathfinder2e_stats.Damage.expand`. You can define an {class}`~pathfinder2e_stats.ExpandedDamage` by initialising the class directly or by adding it to {class}`~pathfinder2e_stats.Damage`. For example, let's define a {prd_equipment}`Flaming <2838>` rune:

In [ ]:
flaming_rune = pf2.Damage("fire", 1, 6) + {
    pf2.DoS.critical_success: [pf2.Damage("fire", 1, 10, persistent=True)]
}
flaming_rune

Our rogue is getting an upgrade! Note that adding `Damage` + `ExpandedDamage` always expands the `Damage` first, so you'll no longer read *deadly d8* but the full success/critical success outcome for it.

In [ ]:
flaming_rapier = rapier + sneak_attack + flaming_rune
flaming_rapier

An {class}`~pathfinder2e_stats.ExpandedDamage` is just a fancy mapping of lists of {class}`~pathfinder2e_stats.Damage`, so usual mapping conversion techniques work:

In [ ]:
dict(flaming_rapier)

## Rolling damage

Now that we have the output of a {func}`~pathfinder2e_stats.check`, like an attack roll or a saving throw, and the {class}`~pathfinder2e_stats.Damage` profile, we can finally roll some {func}`~pathfinder2e_stats.damage`.

Let's reuse the strike outcome from above to roll damage for the flaming rapier:

In [ ]:
flaming_rapier_damage = pf2.damage(strike, flaming_rapier)
flaming_rapier_damage

{func}`~pathfinder2e_stats.damage` makes a copy of the dataset from the check outcome and adds variables to it. Again, most times we're going to care only about `total_damage`, but it can be interesting to understand how we got there:

- **direct_damage** how much immediate, simple damage we got on each of the 100,000 attacks. This is broken down by `damage_type` between piercing, precision and fire.
- **persistent_damage** persistent fire damage caused by the Flaming rune on critical hits. This is rolled by default for 3 rounds, after which we assume either that the target expired, the combat ended, or the persistent damage ended on its own.
- **persistent_damage_DC** DC for persistent damage to end on its own each round.
- **persistent_damage_check** the outcome of the flat check at the end of each of the 3 rounds to end the persistent damage from continuing into the next round.
- **apply_persistent_damage** whether the persistent_damage is still ongoing in this round or it already ended thanks to a successful save on a previous round.
- **total_damage** the sum of direct damage, persistent damage over all the rounds, and splash damage over multiple targets, with the damage type squashed.

{func}`~pathfinder2e_stats.damage` also supports defining weaknesses, resistances, and immunities.

From here we can start dicing and slicing with standard data science techniques. For example, let's plot the damage distribution:

In [ ]:
flaming_rapier_damage.total_damage.to_pandas().hist(
    bins=flaming_rapier_damage.total_damage.max().item() + 1
)

The above clearly shows the three distributions depending on the outcome of the attack roll:
- **Miss** and **Critical Miss** no damage
- **Hit** 4d6+3
- **Critical Hit** (4d6+3)x2 + 1d8 + 1d10 persistent over up to 3 rounds

Let's exclude misses:

In [ ]:
rapier_hit_dmg = flaming_rapier_damage.total_damage[
    flaming_rapier_damage.outcome >= pf2.DoS.success
]
rapier_hit_dmg.min().item()

In [ ]:
rapier_hit_dmg.to_pandas().hist(bins=rapier_hit_dmg.max().item())

Let's do the same for the fireball and let's observe the 4 intersecting distributions for the different saving throw outcomes:
- **Critical Success** no damage
- **Success** (6d6)/2
- **Failure** 6d6
- **Critical Failure** (6d6)x2

In [ ]:
fireball_damage = pf2.damage(save, fireball)
fireball_damage.total_damage.to_pandas().hist(
    bins=fireball_damage.total_damage.max().item() + 1
)

## Multiple targets and variant situations

You may ask yourself, *"What if the same fireball hits multiple targets?"*

TODO

## Conditional buffs/debuffs

TODO

## Armory and tables

For the sake of convenience, we don't need to write by hand the rogue's *+1 Striking Flaming Rapier* every time. {doc}`pf2.armory <../armory>` offers a wealth of weapons, runes, spells, and common class features:

In [ ]:
(
    pf2.armory.swords.rapier(dice=2)
    + pf2.armory.runes.flaming()
    + pf2.armory.class_features.sneak_attack(level=5)
)

We don't need to calculate our rogue's attack bonus either. {doc}`pf2.tables.PC <../tables>` offers a wealth of precalculated progressions over 20 levels over the most common character builds:

In [ ]:
pf2.tables.PC

Each table has a `level` dimension, plus variables and extra dimensions depending on the table:

In [ ]:
pf2.tables.PC.weapon_proficiency.to_pandas()

We can build the attack bonus of our rogue by picking what we want from the PC tables:

In [ ]:
rogue_atk_bonus = (
    # Start with DEX+4 at level 1 and always increase it
    pf2.tables.PC.ability_bonus.boosts.sel(initial=4, drop=True)
    # Get an Apex item at level 17 for +1 DEX
    + pf2.tables.PC.ability_bonus.apex
    # Upgrade weapons as soon as possible: +1 at level 2, +2 at level 10, etc.
    + pf2.tables.PC.attack_item_bonus.potency_rune
    # Trained (+2) at level 1, Expert (+4) at level 5, Master (+6) at level 13
    + pf2.tables.PC.weapon_proficiency.martial
    # Add level to proficiency
    + pf2.tables.PC.level
)
rogue_atk_bonus.to_pandas()

So our level 5 rogue will have an attack bonus of

In [ ]:
rogue_atk_bonus.sel(level=5).item()

Note that the above is just a *typical baseline*, and does not take into consideration buffs, debuffs, suboptimal equipment, or uncommon character progression choices.

There are more {doc}`../tables` available:

- `pf2.tables.DC` is the {prd_rules}`Level-based DCs <2629>` from the GM Core;
- `pf2.tables.EARN_INCOME` is the {prd_skills}`Earn Income <21>` from the Player Core.
- `pf2.tables.NPC` gives you the tables from the {prd_rules}`Building Creatures <2874>` chapter of the GM Core;
- `pf2.tables.SIMPLE_NPC <../tables>` gives you a simplified version of `NPC` with just three targets to blast
  with your attack and spells (or to get blasted by):
  - a weak minion of your level - 2 with all stats rated Low;
  - a worthy foe of your level with all stats rated Moderate, and
  - a boss of your level + 2 with all stats rated High.

In [ ]:
# One very easy, one average and one very hard enemy at level 5
pf2.tables.SIMPLE_NPC.sel(level=5, limited=True).to_pandas().T

## Next steps

Congratulations, you finished the basic tutorial!

From here, you can go look at the {doc}`index`.
In the {doc}`../api`, you will find many functions, flags and options that were omitted here for the sake of brevity.